<a href="https://colab.research.google.com/github/ShirinPullot/podcast_summarization/blob/main/Podify_summarization_of_podcasts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Mar 16 11:51:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
! pip uninstall whisper
! pip install openai-whisper==20230117

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 15.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230117-py3-none-any.whl size=1178629 sha256=f135761889b0b2b41a8a74242998b74b4ee3bff715ef91d1fa70db1411faa9bf
  Stored in directory: /root/.cache/pip/wheels/a5/7d/58/e8e33d698ba51fa30cf9e5473dec24c56bd4893eb4927edc25
Successfully built openai-whisper


In [2]:
import re
import whisper
from whisper.utils import write_vtt
import requests
import os
import nltk
import heapq
nltk.download('punkt')
nltk.download('stopwords')




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True


# 1. First step is to get transcripts from podcast link


In [3]:


model = whisper.load_model("medium")

def get_transcripts(link):
  content = requests.get(link)
  podcast_url = re.findall("(?P<url>\;https?://[^\s]+)", content.text)[0].split(';')[1]
  print(podcast_url)
  

  download = requests.get(podcast_url)

  with open('podcast.mp3', 'wb') as f:
    f.write(download.content)

  result = model.transcribe('podcast.mp3')

  with open('sub.vtt', "w") as txt:
    write_vtt(result["segments"], file=txt)

  return (result['text'], 'sub.vtt')

  




100%|██████████████████████████████████████| 1.42G/1.42G [00:10<00:00, 143MiB/s]


In [5]:
transcripts= get_transcripts('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')
transcript= ''.join(transcripts)

https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEnzoRomero_2022U.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2F1ebf1923-1f21-455d-9520-2319f2b9f17c%2FEnzoRomero_2022U_VO_Intro.mp3



# 2.Summarizing the transcripts obtained from the audio files



1.  Summarization technique using word frequency and NLTK library:



In [6]:
def get_nltk_summary(link):
  
  transcript= get_transcripts(link)
  transcript= ''.join(transcript)
  sentence_list = nltk.sent_tokenize(transcript)
  stopwords = nltk.corpus.stopwords.words('english')
  word_frequencies = {}
  for word in nltk.word_tokenize(transcript):
      if word not in stopwords:
          if word not in word_frequencies.keys():
              word_frequencies[word] = 1
          else:
              word_frequencies[word] += 1
  maximum_frequncy = max(word_frequencies.values())
  for word in word_frequencies.keys():
      word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
  sentence_scores = {}
  for sent in sentence_list:
      for word in nltk.word_tokenize(sent.lower()):
          if word in word_frequencies.keys():
              if len(sent.split(' ')) < 30:
                  if sent not in sentence_scores.keys():
                      sentence_scores[sent] = word_frequencies[word]
                  else:
                      sentence_scores[sent] += word_frequencies[word]
  from pprint import pprint
  summary_sentences = heapq.nlargest(15, sentence_scores, key=sentence_scores.get)
  summary = ' '.join(summary_sentences)
  # pprint( summary)

  print('lines in summary: ', len(summary.split('.')))

2. Summarization using genism(Text rank algorithm)




In [7]:
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords

In [15]:
def get_textrank_summary(link):
  transcript= get_transcripts(link)
  transcript= ''.join(transcript)
  return (summarize(transcript, ratio=0.4))


In [ ]:
x= get_textrank_summary('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')
x

3. Summarization using LSA :
Latent Semantic Analysis is a unsupervised learning algorithm that can be used for extractive text summarization.

It extracts semantically significant sentences by applying singular value decomposition(SVD) to the matrix of term-document frequency.

In [ ]:
! pip install sumy

In [5]:
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser


In [6]:
def get_lsa_summary(link):
  transcript= get_transcripts(link)
  transcript= ''.join(transcript)
  parser=PlaintextParser.from_string(transcript,Tokenizer('english'))
    # creating the summarizer
  lsa_summarizer=LsaSummarizer()
  lsa_summary= lsa_summarizer(parser.document,32)

  # Printing the summary
  for sentence in lsa_summary:
      print(sentence)



In [7]:
x= get_lsa_summary('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')
x

https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEnzoRomero_2022U.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2F1ebf1923-1f21-455d-9520-2319f2b9f17c%2FEnzoRomero_2022U_VO_Intro.mp3
Prosthetics have advanced so much in recent years, but for amputees around the world, they're still often way too expensive.
In his talk from TED 2022, bionic innovator Enzo Romero shares how he's found unconventional solutions to get more advanced prosthetics to the people who need them.
Every week, I interview some of my favorite thinkers to learn how we can bring out the best in ourselves and others.
Find Rethinking on Apple podcasts, Spotify, Amazon music, or wherever you listen.
One in seven people around the world has some kind of disability.
Being born with on my right hand has defined who I am over the past 30 years.
I was fortunate to g

4. Summarization using LexRank:

approach of LexRank is that a particular sentence is recommended by other similar sentences and hence is ranked higher.

Higher the rank, higher is the priority of being included in the summarized text.


In [8]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer


In [9]:
def get_lexrank_summary(link):
  transcript= get_transcripts(link)
  transcript= ''.join(transcript)
  parser=PlaintextParser.from_string(transcript,Tokenizer('english'))
    # Creating a summary of 3 sentences.
  lex_rank_summarizer = LexRankSummarizer()
  lexrank_summary = lex_rank_summarizer(parser.document,sentences_count=32)

  # Printing the summary
  for sentence in lexrank_summary:
    print(sentence)

In [10]:
x= get_lexrank_summary('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')
x

https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEnzoRomero_2022U.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2F1ebf1923-1f21-455d-9520-2319f2b9f17c%2FEnzoRomero_2022U_VO_Intro.mp3
It's TED Talks Daily.
Prosthetics have advanced so much in recent years, but for amputees around the world, they're still often way too expensive.
He makes them more affordable and functional.
Most of you aren't just listening right now.
But what if you could be saving money by switching to Progressive?
Progressive Casualty Insurance Company and Affiliates national average 12-month savings of $698 by new customers surveyed who saved with Progressive between June 2021 and May 2022.
Discounts not available in all states and situations.
From then on, no greater purpose lived in my mind, which was to have a prosthesis like Luke Skywalker.
I was able

5. Summarization using Luhn:

Luhn Summarization algorithm’s approach is based on TF-IDF (Term Frequency-Inverse Document Frequency). It is useful when very low frequent words as well as highly frequent words(stopwords) are both not significant.

In [13]:
# Import the summarizer
from sumy.summarizers.luhn import LuhnSummarizer
# Creating the parser
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser

In [19]:
def get_luhn_summary(link):
  transcript= get_transcripts(link)
  transcript= ''.join(transcript)
  parser=PlaintextParser.from_string(transcript,Tokenizer('english'))
      #  Creating the summarizer
  luhn_summarizer=LuhnSummarizer()
  luhn_summary=luhn_summarizer(parser.document,sentences_count=32)

  # Printing the summary
  for sentence in luhn_summary:
    print(sentence)

In [20]:
x= get_luhn_summary('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')
x

https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEnzoRomero_2022U.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2F1ebf1923-1f21-455d-9520-2319f2b9f17c%2FEnzoRomero_2022U_VO_Intro.mp3
Prosthetics have advanced so much in recent years, but for amputees around the world, they're still often way too expensive.
In his talk from TED 2022, bionic innovator Enzo Romero shares how he's found unconventional solutions to get more advanced prosthetics to the people who need them.
Drivers who save by switching save nearly $700 on average, and auto customers qualify for an average of seven discounts.
Progressive Casualty Insurance Company and Affiliates national average 12-month savings of $698 by new customers surveyed who saved with Progressive between June 2021 and May 2022.
One in seven people around the world has some kind of disabil

6. Summarization using KL-Sum:

Another extractive method is the KL-Sum algorithm.

It selects sentences based on similarity of word distribution as the original text. It aims to lower the KL-divergence criteria (learn more). It uses greedy optimization approach and keeps adding sentences till the KL-divergence decreases.

In [16]:
from sumy.summarizers.kl import KLSummarizer
# Creating the parser
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser


In [21]:
def get_klsum_summary(link):
  transcript= get_transcripts(link)
  transcript= ''.join(transcript)
  parser=PlaintextParser.from_string(transcript,Tokenizer('english'))
  # Instantiating the  KLSummarizer
  kl_summarizer=KLSummarizer()
  kl_summary=kl_summarizer(parser.document,sentences_count=32)

  # Printing the summary
  for sentence in kl_summary:
      print(sentence)

In [22]:
x= get_klsum_summary('https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkczIuZmVlZGJ1cm5lci5jb20vdGVkdGFsa3NfYXVkaW8/episode/ZW4uYXVkaW8udGFsay50ZWQuY29tOjk3NDA4?sa=X&ved=0CAUQkfYCahgKEwiQuOnlr7L6AhUAAAAAHQAAAAAQiwc')
x

https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEnzoRomero_2022U.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2F1ebf1923-1f21-455d-9520-2319f2b9f17c%2FEnzoRomero_2022U_VO_Intro.mp3
It's TED Talks Daily.
Support for TED Talks Daily comes from Progressive.
Multitask right now.
Quote today at progressive.com.
Potential savings will vary.
One in seven people around the world has some kind of disability.
Since this was a research project, we had enough funding to buy stuff from abroad.
This is one of the main reasons why it will cost at least $40,000 to make this prosthesis.
But $40,000 is way too much, considering that four out of five people with upper limb amputation live in developing countries.
Even the $25,000 that costs a robotic prosthesis imported from abroad is way too much.
In Peru, that is equivalent to work full ti